In [1]:
import os


os.environ['SPARK_NAME'] = "/opt/spark"
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, sum, count


spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/16 16:40:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/16 16:40:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local[2] appName=Create-DataFrame>

**Zadanie 1**  
Na zbiorze danych '_Recipe Reviews ..._' wykonaj:  
1.1  Zmień nazwę pierwszej kolumny z `_c0` na `id`.  

In [3]:
sc = spark.sparkContext
df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",")
df_reviews.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- reply_count: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- thumbs_down: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- best_score: string (nullable = true)
 |-- text: string (nullable = true)



In [4]:
df_reviews = df_reviews.withColumnRenamed('_c0', 'id')
df_reviews.printSchema()

root
 |-- id: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- reply_count: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- thumbs_down: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- best_score: string (nullable = true)
 |-- text: string (nullable = true)



1.2  Wyświetl 10 najwyższych wartości w kolumnie `reply_count`.

In [5]:
top_reply_count = df_reviews.select('reply_count').orderBy(desc('reply_count')).limit(10)
print('10 najwyższych wartości w kolumnie reply_count:')
top_reply_count.show()

10 najwyższych wartości w kolumnie reply_count:
+-----------+
|reply_count|
+-----------+
|          3|
|          3|
|          3|
|          3|
|          3|
|          3|
|          2|
|          2|
|          2|
|          2|
+-----------+



1.3  Wyświetl 10 najwyższych sum wartości w kolumnie `best_score` dla każdego przepisu (grupowanie).  

In [6]:
top_best_score = df_reviews.groupBy('recipe_name') \
    .agg(sum('best_score').alias('total_best_score')) \
    .orderBy(desc('total_best_score')) \
    .limit(10)
print('10 najwyższych sum wartości w kolumnie best_score:')
top_best_score.show()

10 najwyższych sum wartości w kolumnie best_score:
+--------------------+----------------+
|         recipe_name|total_best_score|
+--------------------+----------------+
|   Cheeseburger Soup|         98863.0|
|  Creamy White Chili|         85497.0|
|Amish Breakfast C...|         64880.0|
|Best Ever Banana ...|         64247.0|
|Favorite Chicken ...|         60755.0|
|Basic Homemade Bread|         59867.0|
|Flavorful Chicken...|         59195.0|
|Zucchini Pizza Ca...|         54032.0|
|Enchilada Casser-...|         51975.0|
|    Cauliflower Soup|         47905.0|
+--------------------+----------------+



1.4  Które 10 przepisów miało najwięcej komentarzy?  

In [7]:
top_comments = df_reviews.groupBy('recipe_name') \
    .agg(count('comment_id').alias('total_comments')) \
    .orderBy(desc('total_comments')) \
    .limit(10)
print('10 przepisów z największą liczbą komentarzy:')
top_comments.show()

10 przepisów z największą liczbą komentarzy:
+--------------------+--------------+
|         recipe_name|total_comments|
+--------------------+--------------+
|   Cheeseburger Soup|           725|
|  Creamy White Chili|           654|
|Best Ever Banana ...|           509|
|Enchilada Casser-...|           421|
|Basic Homemade Bread|           397|
|Favorite Chicken ...|           395|
|Flavorful Chicken...|           368|
|Amish Breakfast C...|           338|
|Zucchini Pizza Ca...|           332|
|    Cauliflower Soup|           324|
+--------------------+--------------+



1.5  Wyświetl rozkład wartości w kolumnie `stars`.

In [8]:
stars_distribution = df_reviews.groupBy('stars').count().orderBy(desc('stars'))
print('Rozkład wartości w kolumnie stars:')
stars_distribution.show()

Rozkład wartości w kolumnie stars:
+-----+-----+
|stars|count|
+-----+-----+
|    5|13829|
|    4| 1655|
|    3|  490|
|    2|  232|
|    1|  280|
|    0| 1696|
| NULL|   86|
+-----+-----+



**Zadanie 2**  
Wczytaj zbiór danych `employee` nakazując Sparkowi wywnioskowanie bardziej optymalnych typów danych niż domyślny typ `string`.

In [9]:
employee_df = spark.read.csv('data/employee.csv', header=True, inferSchema=True)

employee_df.printSchema()
employee_df.show(5)

[Stage 12:================================================>         (5 + 1) / 6]

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: double (nullable = true)

+---+----------+----------+---+-------+
| id| firstname|  lastname|age| salary|
+---+----------+----------+---+-------+
|  1|  Wojciech|Malinowski| 33|9301.66|
|  2|      Adam|     Pysla| 38|8202.74|
|  3|Aleksandra|  Barański| 37|7149.76|
|  4|      Adam|    Szczaw| 35|7585.43|
|  5|Aleksandra|  Kowalski| 46|7260.76|
+---+----------+----------+---+-------+
only showing top 5 rows



**Zadanie 3**  
Jaki jest czas wykonania operacji `df.filter(df["salary"] > 10000).count()` tym razem przy numerycznym typie kolumny `salary`? Jest jakaś różnica?

In [10]:
%%time
employee_df.filter(employee_df['salary'] > 10000).count()

[Stage 14:================================================>         (5 + 1) / 6]

CPU times: user 9.71 ms, sys: 0 ns, total: 9.71 ms
Wall time: 8.85 s


314703

Czas operacji wydłużył się o około sekundę.

**Zadanie 4**  
Wykorzystując przykład z dokumentacji klasy `Bucketizer` (https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Bucketizer.html) podziel dane w kolumnie `age` zbioru `employee` na buckety co 10 lat (10-19, 20-29, ..., 60-69) i wyświetl te dane dla 20 pierwszych wierzy w formie surowej oraz całość grupując po bucketach i licząc ile osób znalazło się w każdym z nich.

In [11]:
from pyspark.ml.feature import Bucketizer

splits = [10, 20, 30, 40, 50, 60, 70]

bucketizer = Bucketizer(
    splits=splits,
    inputCol='age',
    outputCol='age_bucket'
)

bucketed_df = bucketizer.transform(employee_df)

print('Dane surowe z bucketyzowaną kolumną:')
bucketed_df.select('age', 'age_bucket').show(20)

grouped_buckets = bucketed_df.groupBy('age_bucket').agg(count('*').alias('count'))

print('Liczba osób w każdym buckecie:')
grouped_buckets.orderBy('age_bucket').show()

Dane surowe z bucketyzowaną kolumną:
+---+----------+
|age|age_bucket|
+---+----------+
| 33|       2.0|
| 38|       2.0|
| 37|       2.0|
| 35|       2.0|
| 46|       3.0|
| 64|       5.0|
| 18|       0.0|
| 26|       1.0|
| 43|       3.0|
| 63|       5.0|
| 18|       0.0|
| 68|       5.0|
| 34|       2.0|
| 64|       5.0|
| 25|       1.0|
| 46|       3.0|
| 38|       2.0|
| 53|       4.0|
| 59|       4.0|
| 25|       1.0|
+---+----------+
only showing top 20 rows

Liczba osób w każdym buckecie:


[Stage 18:================================================>         (5 + 1) / 6]

+----------+-------+
|age_bucket|  count|
+----------+-------+
|       0.0| 783516|
|       1.0|3919857|
|       2.0|3921865|
|       3.0|3921239|
|       4.0|3924423|
|       5.0|3529100|
+----------+-------+

